# 具体应用：用Paddlepaddle做一个服装广告文案生成器

## 1. 安装相关库

In [1]:
# 检查paddlepaddle是否可以使用GPU
import paddle
paddle.utils.run_check()

Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


按照数据集要求对其进行整理，格式为“序号\t输入文本\t标签”

In [2]:
import pandas as pd

df = pd.read_excel("data.xlsx")
Keys = df["关键词"].values
Txts = df["文案"].values

with open("format_data.txt", "w",encoding='utf-8') as f:
    for i, k in enumerate(Keys):
        t = Txts[i]
        f.write("{}\t{}\t{}\n".format(i, k, t))

## 2. 调用Paddlehub模型进行预训练

### 模型我们选用ERNIE-GEN模型

论文地址：https://arxiv.org/abs/2001.11314

![](https://ai-studio-static-online.cdn.bcebos.com/9698f8fef8004f579d01ca0d4a8a8255d58a8cc895c44d59b97141c82a23b833)


In [3]:
import paddlehub as hub

module = hub.Module(name="ernie_gen")

result = module.finetune(
    train_path='format_data.txt',
    save_dir="nlg_param",
    max_steps=1500,
    noise_prob=0,
    save_interval=500,
    batch_size=16,
    max_encode_len=100,
    max_decode_len=100,
    learning_rate=5e-5
)

# 将训练参数打包为hub model
module.export(params_path=result['last_save_path'], module_name="nlg", author="etta")
!hub install nlg

E:\anaconda3\envs\paddle_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E:\anaconda3\envs\paddle_env\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-12-30 20:08:39,263] [    INFO] - Already cached C:\Users\11409\.paddlenlp\models\ernie-1.0\vocab.txt
[2023-12-30 20:08:39,280] [    INFO] - tokenizer config file saved in C:\Users\11409\.paddlenlp\models\ernie-1.0\tokenizer_config.json
[2023-12-30 20:08:39,281] [    INFO] - Special tokens file saved in C:\Users\11409\.paddlenlp\models\ernie-1.0\special_tokens_map.json
[2023-12-30 20:08:39,325] [    INFO] - Already cached C:\Users\11409\.paddlenlp\models\ernie-1.0\ernie_v1_chn_base.pdparams
[2023-12-30 20:08:39,327] [   DEBUG] - init ErnieModel with config

## 3. 运行预测

In [4]:
import paddlehub as hub

module = hub.Module(directory="nlg")

[2023-12-30 21:26:29,956] [    INFO] - Already cached C:\Users\11409\.paddlenlp\models\ernie-1.0\ernie_v1_chn_base.pdparams
[2023-12-30 21:26:29,956] [   DEBUG] - init ErnieModel with config: {'attention_probs_dropout_prob': 0.1, 'hidden_act': 'relu', 'hidden_dropout_prob': 0.1, 'hidden_size': 768, 'initializer_range': 0.02, 'max_position_embeddings': 513, 'num_attention_heads': 12, 'num_hidden_layers': 12, 'type_vocab_size': 2, 'vocab_size': 18000, 'pad_token_id': 0}
[2023-12-30 21:26:30,315] [    INFO] - loading pretrained model from C:\Users\11409\.paddlenlp\models\ernie-1.0\ernie_v1_chn_base.pdparams
[2023-12-30 21:26:31,864] [    INFO] - param:mlm_bias not set in pretrained model, skip
[2023-12-30 21:26:31,864] [    INFO] - param:mlm.weight not set in pretrained model, skip
[2023-12-30 21:26:31,864] [    INFO] - param:mlm.bias not set in pretrained model, skip
[2023-12-30 21:26:31,864] [    INFO] - param:mlm_ln.weight not set in pretrained model, skip
[2023-12-30 21:26:31,870] [  

In [13]:
test_texts = ["宽松"]
results = module.generate(texts=test_texts, use_gpu=False, beam_width=10)
for result in results[0]:
    print(result)

这款连衣裙采用了宽松的版型设计，能够很好的修饰身材，更好的修饰身材，更好的修饰了腿部的线条，让你轻松
这款连衣裙采用了宽松的版型设计，能够很好的修饰身材，更好的修饰身材比例，更好的修饰了腿部的线条，让你
这款连衣裙采用了宽松的版型设计，能够很好的修饰身材，更好的修饰身材，更好的修饰了腿部的线条，更好的修
这款连衣裙采用了宽松的版型设计，能够很好的修饰身材，更好的修饰身材比例，更好的修饰了腿部的线条，同时
这款连衣裙采用了宽松的版型设计，能够很好的修饰身材，更好的修饰身材比例，更好的修饰了腿部的线条，更好
这款连衣裙采用了宽松的版型设计，能够很好的修饰身材，更好的修饰身材比例，更好的修饰身材比例，更好的修
这款连衣裙采用了宽松的版型设计，能够很好的修饰身材，更好的修饰身材比例，更好的修饰身材，更好的修饰身
这款连衣裙采用了宽松的版型设计，能够很好的修饰腿部的线条，更好的修饰身材比例，更好的修饰身材比例，更
这款连衣裙采用了宽松的版型设计，能够很好的修饰身材，更好的修饰身材比例，更好的修饰身材比例，更好的遮
这款连衣裙采用了宽松的版型设计，能够很好的修饰身材，更好的修饰身材比例，更好的修饰了腿部的线条，更加


In [6]:
test_texts = ["性感"]
results = module.generate(texts=test_texts, use_gpu=False, beam_width=5)
for result in results[0]:
    print(result)

这款连衣裙采用了v领的设计，修饰脖颈线条，凸显出优美的颈部线条。精致的镂空剪裁，凸显出女性的优雅气质
这款连衣裙采用了v领的设计，修饰脖颈线条，凸显出优美的颈部线条。精致的镂空设计，凸显出女性的优雅气质
这款连衣裙采用了v领的设计，修饰脖颈线条，凸显出优美的颈部线条。精致的镂空剪裁，凸显出女性曼妙的身姿
这款连衣裙采用了v领的设计，修饰脖颈线条，凸显出优美的颈部线条。精致的镂空剪裁，凸显出优雅的女人味。
这款连衣裙采用了v领的设计，修饰脖颈线条，凸显出优美的颈部线条。精致的镂空剪裁，凸显出优雅的女人味，


In [16]:
test_texts = ["简约"]
results = module.generate(texts=test_texts, use_gpu=False, beam_width=5)
for result in results[0]:
    print(result)

这款连衣裙采用优质面料，手感细腻顺滑，垂感好，垂感好，手感顺滑，垂感好，挺括有型，挺括有型，宽松的版
这款连衣裙采用优质面料，手感细腻顺滑，垂感好，垂感好，手感顺滑，垂感好，挺括有型，挺括有型，挺括有型
这款连衣裙采用优质面料，手感细腻顺滑，垂感好，垂感好，手感顺滑，垂感好，挺括有型，挺括有型，有型有型
这款连衣裙采用优质面料，手感细腻顺滑，垂感好，垂感好，手感顺滑，垂感好，挺括有型，挺括有型，宽松版型
这款连衣裙采用优质面料，手感细腻顺滑，垂感好，垂感好，手感顺滑，垂感好，挺括有型，挺括有型，宽松的廓


## 4. 改进方向：

可以看到数据集太少，出现了明显的过拟合现象，可以多搜集一些。